### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
totalSchools = school_data_complete["school_name"].nunique()
totalStudents = school_data_complete["student_name"].count()
totalBudgetPrep = school_data_complete.drop_duplicates(subset=["school_name"])
totalBudget = totalBudgetPrep["budget"].sum()
avgMath = school_data_complete["math_score"].mean()
avgReading = school_data_complete["reading_score"].mean()
percentagePassMath = school_data_complete[school_data_complete["math_score"] >= 70].count()[0] / school_data_complete.count()[0] * 100
percentagePassRead = school_data_complete[school_data_complete["reading_score"] >= 70].count()[0] / school_data_complete.count()[0] * 100
percentagePassRM = (school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)].count()[0] / school_data_complete.count()[0]) * 100
calculatedDF = pd.DataFrame({"Total Schools": [totalSchools],
                             "Total Students": [totalStudents],
                             "Total Budget": [totalBudget],
                             "Average Math Score": [avgMath],
                             "Average Reading Score": [avgReading],
                             "% Passing Math": [percentagePassMath],
                             "% Passing Reading": [percentagePassRead],
                             "% Overall Passing": [percentagePassRM]
                            })
calculatedDF

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
schoolSummDF1 = pd.DataFrame()
groupedDF = school_data_complete.groupby(["school_name","type","budget"])

"""Aggregates total students, average math, and average reading score"""
schoolSummDF1["Total Students"] = groupedDF["student_name"].count()
schoolSummDF1["Average Math Score"] = groupedDF["math_score"].mean()
schoolSummDF1["Average Reading Score"] = groupedDF["reading_score"].mean()

"""Resets type and budget to not be keys. The budget and type values needed to be static."""
schoolSummDF1.reset_index(["type","budget"],inplace=True)

"""Calculates per student budget."""
schoolSummDF1["Per Student Budget"] = schoolSummDF1["budget"] / schoolSummDF1["Total Students"]

"""Creates a new dataframe that only brings in students that have a passing math score."""
passMathDF = school_data_complete[school_data_complete["math_score"] >= 70]
passMathDFgrouped = passMathDF.groupby(["school_name"])
passMathDFgroupedcount = passMathDFgrouped.count()

"""Creates a new dataframe that only brings in students that have a passing reading score."""
passReadDF = school_data_complete[school_data_complete["reading_score"] >= 70]
passReadDFgrouped = passReadDF.groupby(["school_name"])
passReadDFgroupedcount = passReadDFgrouped.count()

"""Creates a new dataframe that only brings in students that are passing both."""
passBothDF = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70),]
passBothDFgrouped = passBothDF.groupby(["school_name"])
passBothDFgroupedcount = passBothDFgrouped.count()

"""Merges the math, reading, and overall passing tables."""
schoolSummDF2 = pd.merge(passMathDFgroupedcount, passReadDFgroupedcount, how="left", on=["school_name"], suffixes=['_math','_read'])
schoolSummDF2columns = schoolSummDF2[["Student ID_math", "Student ID_read"]]
schoolSummDF2columns = schoolSummDF2columns.rename(columns={"Student ID_math": "# Passed Math", "Student ID_read": "# Passed Reading"})

schoolSummDF3 = pd.merge(schoolSummDF2columns, passBothDFgroupedcount, how="left", on=["school_name"], suffixes=['_ssdf2','_pbdf2c'])
schoolSummDF3columns = schoolSummDF3[["# Passed Math", "# Passed Reading", "Student ID"]]
schoolSummDF3columns = schoolSummDF3columns.rename(columns={"Student ID": "# Passed M&R"})

"""Merges the above table with the original aggregation table"""
schoolSummDF4 = pd.merge(schoolSummDF1, schoolSummDF3columns, how="left", on=["school_name"])

"""Cleans up column names"""
schoolSummDF4 = schoolSummDF4.rename(columns={"type": "School Type", "budget": "Total School Budget"})

"""Calculates percentage columns."""
schoolSummDF4["% Passing Math"] = (schoolSummDF4["# Passed Math"] / schoolSummDF4["Total Students"]) * 100
schoolSummDF4["% Passing Reading"] = (schoolSummDF4["# Passed Reading"] / schoolSummDF4["Total Students"]) * 100
schoolSummDF4["% Passing M&R"] = (schoolSummDF4["# Passed M&R"] / schoolSummDF4["Total Students"]) * 100

"""Sets the school key to blank"""
schoolSummDF4.reset_index(["school_name"],inplace=True)
schoolSummDF4 = schoolSummDF4.rename(columns={"school_name": " "})
schoolSummDF4.set_index(" ", inplace=True)

"""Selects only relevant columns"""
schoolSummDF4Final = schoolSummDF4[["Total Students", "Total School Budget", "Per Student Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing M&R"]]
# schoolSummDF4
schoolSummDF4Final


,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing M&R
,,,,,,,,
Bailey High School,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
schoolSummDF4Final = schoolSummDF4Final.sort_values(["% Passing M&R"], ascending=False)
schoolSummDF4Final.head()


,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing M&R
,,,,,,,,
Cabrera High School,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
schoolSummDF4Final = schoolSummDF4Final.sort_values(["% Passing M&R"], ascending=True)
schoolSummDF4Final.head()

,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing M&R
,,,,,,,,
Rodriguez High School,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
mathPer9GradeFiltered = school_data_complete[school_data_complete["grade"] == "9th"]
mathPer10GradeFiltered = school_data_complete[school_data_complete["grade"] == "10th"]
mathPer11GradeFiltered = school_data_complete[school_data_complete["grade"] == "11th"]
mathPer12GradeFiltered = school_data_complete[school_data_complete["grade"] == "12th"]

mathPer9GradeColumns = mathPer9GradeFiltered[["school_name","math_score"]]
mathPer10GradeColumns = mathPer10GradeFiltered[["school_name","math_score"]]
mathPer11GradeColumns = mathPer11GradeFiltered[["school_name","math_score"]]
mathPer12GradeColumns = mathPer12GradeFiltered[["school_name","math_score"]]

mathPer9GradeGrouped = mathPer9GradeColumns.groupby(["school_name"]).mean()
mathPer10GradeGrouped = mathPer10GradeColumns.groupby(["school_name"]).mean()
mathPer11GradeGrouped = mathPer11GradeColumns.groupby(["school_name"]).mean()
mathPer12GradeGrouped = mathPer12GradeColumns.groupby(["school_name"]).mean()

mathPerGradeDFmerged1 = pd.merge(mathPer9GradeGrouped, mathPer10GradeGrouped,on=['school_name'],how='left',suffixes=["_9th","_10th"])
mathPerGradeDFmerged2 = pd.merge(mathPer11GradeGrouped, mathPer12GradeGrouped,on=['school_name'],how='left',suffixes=["_11th","_12th"])
mathPerGradeDFmerged3 = pd.merge(mathPerGradeDFmerged1, mathPerGradeDFmerged2,on=['school_name'],how='left')
mathPerGradeDFmerged3.reset_index(["school_name"],inplace=True)

mathPerGradeDFmerged3 = mathPerGradeDFmerged3.rename(columns={"school_name": " ", "math_score_9th":"9th",
                                                              "math_score_10th": "10th",
                                                              "math_score_11th": "11th",
                                                              "math_score_12th": "12th"})

mathPerGradeDFmerged3.set_index(" ", inplace=True)
mathPerGradeDFmerged3

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
readPer9GradeFiltered = school_data_complete[school_data_complete["grade"] == "9th"]
readPer10GradeFiltered = school_data_complete[school_data_complete["grade"] == "10th"]
readPer11GradeFiltered = school_data_complete[school_data_complete["grade"] == "11th"]
readPer12GradeFiltered = school_data_complete[school_data_complete["grade"] == "12th"]

readPer9GradeColumns = readPer9GradeFiltered[["school_name","reading_score"]]
readPer10GradeColumns = readPer10GradeFiltered[["school_name","reading_score"]]
readPer11GradeColumns = readPer11GradeFiltered[["school_name","reading_score"]]
readPer12GradeColumns = readPer12GradeFiltered[["school_name","reading_score"]]

readPer9GradeGrouped = readPer9GradeColumns.groupby(["school_name"]).mean()
readPer10GradeGrouped = readPer10GradeColumns.groupby(["school_name"]).mean()
readPer11GradeGrouped = readPer11GradeColumns.groupby(["school_name"]).mean()
readPer12GradeGrouped = readPer12GradeColumns.groupby(["school_name"]).mean()

readPerGradeDFmerged1 = pd.merge(readPer9GradeGrouped, readPer10GradeGrouped,on=['school_name'],how='left',suffixes=["_9th","_10th"])
readPerGradeDFmerged2 = pd.merge(readPer11GradeGrouped, readPer12GradeGrouped,on=['school_name'],how='left',suffixes=["_11th","_12th"])
readPerGradeDFmerged3 = pd.merge(readPerGradeDFmerged1, readPerGradeDFmerged2,on=['school_name'],how='left')
readPerGradeDFmerged3.reset_index(["school_name"],inplace=True)

readPerGradeDFmerged3 = readPerGradeDFmerged3.rename(columns={"school_name": " ",
                                                              "reading_score_9th":"9th",
                                                              "reading_score_10th": "10th",
                                                              "reading_score_11th": "11th",
                                                              "reading_score_12th": "12th"})
readPerGradeDFmerged3.set_index(" ", inplace=True)
readPerGradeDFmerged3

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [11]:
spendingBins = [0, 585, 615, 645, 675]
spendingBinLabels = ["<$584","$585-615","$615-645","$645-675"]

"""Aggregates the bin categories"""
schoolSummDF4spendcut = pd.cut(schoolSummDF4["Per Student Budget"], spendingBins, labels=spendingBinLabels, right=True)
schoolSummDF5 = pd.merge(schoolSummDF4, schoolSummDF4spendcut, how="left", on=" ")

'''Averages the math score, reading score, and per student budget.'''
schoolSummDF5grouped = schoolSummDF5.groupby(["Per Student Budget_y"])
schoolSummDF5average = schoolSummDF5grouped[["Per Student Budget_x", "Average Math Score", "Average Reading Score"]].mean()

"""This aggregates % passing Math, Reading, and Overall Passing Rate"""
schoolSummDF5studentssum = schoolSummDF5grouped[["Total Students","# Passed Math", "# Passed Reading", "# Passed M&R"]].sum()
schoolSummDF5studentssum["% Passing Math"] = (schoolSummDF5studentssum["# Passed Math"] / schoolSummDF5studentssum["Total Students"]) * 100
schoolSummDF5studentssum["% Passing Reading"] = (schoolSummDF5studentssum["# Passed Reading"] / schoolSummDF5studentssum["Total Students"]) * 100
schoolSummDF5studentssum["Overall Passing"] = (schoolSummDF5studentssum["# Passed M&R"] / schoolSummDF5studentssum["Total Students"]) * 100

"""Selects relevant columns"""
schoolSummDF5studentsummcolumns = schoolSummDF5studentssum[["% Passing Math", "% Passing Reading", "Overall Passing"]]
schoolSummDF5averagecolumns = schoolSummDF5average[["Average Math Score", "Average Reading Score"]]

"""Joins final table"""
schoolSummDF5final = pd.merge(schoolSummDF5averagecolumns, schoolSummDF5studentsummcolumns, how='left', on=["Per Student Budget_y"]) 

"""Final Formatting"""
schoolSummDF5final.reset_index(inplace=True)
schoolSummDF5final = schoolSummDF5final.rename(columns={"Per Student Budget_y":" "})
schoolSummDF5final = schoolSummDF5final.set_index([" "])
schoolSummDF5final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing
,,,,,
<$584,83.455399,83.933814,93.702889,96.686558,90.640704
$585-615,83.599686,83.885211,94.124128,95.886889,90.121190
$615-645,79.079225,81.891436,71.400428,83.614770,60.289317
$645-675,76.997210,81.027843,66.230813,81.109397,53.528791


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
sizeBins = [0 , 1000, 2000, 5000]
sizeBinLabels = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

"""Aggregates the bin categories"""
schoolSummDF4sizecut = pd.cut(schoolSummDF4["Total Students"], sizeBins, labels=sizeBinLabels, right=True)
schoolSummDF6 = pd.merge(schoolSummDF4, schoolSummDF4sizecut, how="left", on=" ")

'''Averages the math score, reading score, and per student budget.'''
schoolSummDF6grouped = schoolSummDF6.groupby(["Total Students_y"])
schoolSummDF6average = schoolSummDF6grouped[["Average Math Score", "Average Reading Score"]].mean()

"""This aggregates % passing Math, Reading, and Overall Passing Rate"""
schoolSummDF6studentssum = schoolSummDF6grouped[["Total Students_x", "# Passed Math", "# Passed Reading", "# Passed M&R"]].sum()
schoolSummDF6studentssum["% Passing Math"] = (schoolSummDF6studentssum["# Passed Math"] / schoolSummDF6studentssum["Total Students_x"]) * 100
schoolSummDF6studentssum["% Passing Reading"] = (schoolSummDF6studentssum["# Passed Reading"] / schoolSummDF6studentssum["Total Students_x"]) * 100
schoolSummDF6studentssum["Overall Passing"] = (schoolSummDF6studentssum["# Passed M&R"] / schoolSummDF6studentssum["Total Students_x"]) * 100

"""Selects relevant columns"""
schoolSummDF6studentsummcolumns = schoolSummDF6studentssum[["% Passing Math", "% Passing Reading", "Overall Passing"]]
schoolSummDF6averagecolumns = schoolSummDF6average[["Average Math Score", "Average Reading Score"]]

"""Joins final table"""
schoolSummDF6final = pd.merge(schoolSummDF6averagecolumns, schoolSummDF6studentsummcolumns, how='left', on=["Total Students_y"]) 

"""Final Formatting"""
schoolSummDF6final.reset_index(inplace=True)
schoolSummDF6final = schoolSummDF6final.rename(columns={"Total Students_y":" "})
schoolSummDF6final = schoolSummDF6final.set_index([" "])
schoolSummDF6final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing
,,,,,
Small (<1000),83.821598,83.929843,93.952484,96.040317,90.136789
Medium (1000-2000),83.374684,83.864438,93.616522,96.773058,90.624267
Large (2000-5000),77.746417,81.344493,68.652380,82.125158,56.574046


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
schoolSummDF7 = schoolSummDF4

'''Averages the math score, reading score, and per student budget.'''
schoolSummDF7grouped = schoolSummDF7.groupby(["School Type"])
schoolSummDF7average = schoolSummDF7grouped[["School Type", "Average Math Score", "Average Reading Score"]].mean()

"""This aggregates % passing Math, Reading, and Overall Passing Rate"""
schoolSummDF7studentssum = schoolSummDF7grouped[["Total Students","# Passed Math", "# Passed Reading", "# Passed M&R"]].sum()
schoolSummDF7studentssum["% Passing Math"] = (schoolSummDF7studentssum["# Passed Math"] / schoolSummDF7studentssum["Total Students"]) * 100
schoolSummDF7studentssum["% Passing Reading"] = (schoolSummDF7studentssum["# Passed Reading"] / schoolSummDF7studentssum["Total Students"]) * 100
schoolSummDF7studentssum["Overall Passing"] = (schoolSummDF7studentssum["# Passed M&R"] / schoolSummDF7studentssum["Total Students"]) * 100

"""Selects relevant columns"""
schoolSummDF7studentsummcolumns = schoolSummDF7studentssum[["% Passing Math", "% Passing Reading", "Overall Passing"]]

"""Joins final table"""
schoolSummDF7final = pd.merge(schoolSummDF7average, schoolSummDF7studentsummcolumns, how='left', on=["School Type"]) 

schoolSummDF7final

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.701821,96.645891,90.560932
District,76.956733,80.966636,66.518387,80.905249,53.695878
